Data generation

In [1]:
import numpy as np

# add the path to the abcTau package
import sys
#sys.path.append('./abcTau')
sys.path.append('C:\\Users\\ipochino\\AppData\\Local\\anaconda3\\envs\\isttc\\Lib\\site-packages\\abcTau') # IP: replaced previous line with that; relative path was not working
import abcTau
from scipy.optimize import curve_fit
from statsmodels.tsa.stattools import acf

import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
#from calculate_tau import fit_single_exp, func_single_exp_monkey
from cfg_global import project_folder_path
os.chdir(current_wd)

### Generate time series

In [2]:
save_folder = project_folder_path + 'results\\synthetic_data\\dataset\\'

n_signals = 1

In [31]:
tau = np.array([100])
D = 1/tau
deltaT = 1 # 1ms
T = 1000
numTrials = 1

data_mean = 0.002 # average of firing rate
data_var = 0.003 # variance of firing rate

binSize = 50 # bins size for binning the data for calculating acf
binsData =  np.arange(0, T + binSize, binSize)
numBinData = len(binsData)-1
#print(binsData)
print(numBinData)

# num_lags = 20
# maxTimeLag = int(T/binSize)
# print(maxTimeLag)

20


In [36]:
for signal_ in range(n_signals):
    ###############
    # OU process 
    ou = abcTau.OU_gen(tau, D, deltaT, T, numTrials)
    print('ou.shape: ', ou.shape)
    
    # ou_check = np.max(ou)
    # print(ou_check)
    # # adjust OU process - scale, shift 
    # ou_std = np.sqrt(data_var)
    # ou_mean = data_mean
    # ou_all = ou_std * ou + ou_mean # this is scale and shift, where is rectify? - no rectify here, this will come for the Poisson rate
    # print('ou_all.shape: ', ou_all.shape)
    
    ###############
    # One tau UO with Poisson spike count
    ou_std =  np.sqrt(data_var - data_mean)/deltaT # law of total variance  
    ou_mean = data_mean/deltaT # law of total expectation
    
    # fit mean and var
    ou_poisson = ou_std * ou + ou_mean
    ou_poisson[ou_poisson < 0] = 0 # rectifying
    
    # bin rate and generate spikes
    # numBin = int(T/deltaT)
    ou_poisson_rate_ = abcTau.binData(ou_poisson, [numTrials,numBinData]) * deltaT
    ou_poisson_rate = np.random.poisson(ou_poisson_rate_)
    
    # ###############
    # # Going from rate to spikes
    # spikeTrain = np.zeros(ou_poisson_rate.shape)
    # print(spikeTrain.shape)
    
    # for i in range(ou_poisson_rate.shape[0]):
    #     spikeTrain[i,:] = [1 if ou_poisson_rate[i,j] > np.random.random() else 0 for j in range(ou_poisson_rate.shape[1])]
    
    # spike_times = np.where(np.squeeze(spikeTrain) == 1)[0]
    # print('N spikes {}'.format(spike_times.shape))
    
    # # bin for ACF calculation
    # ou_spiketrain_binned = abcTau.binData(spikeTrain, [numTrials, numBinData]) * deltaT
    # print(ou_spiketrain_binned.shape)

    # ###############
    # Going from rate to spikes - Poisson thinning
    binary_spikeTrain = (np.random.random(ou_poisson_rate.shape) < ou_poisson_rate / np.max(ou_poisson_rate)).astype(int)

    ou_spiketrain_binned_thinned = abcTau.binData(binary_spikeTrain, [numTrials, numBinData]) * deltaT
    print(ou_spiketrain_binned_thinned.shape)

    
    # # store
    # np.save(save_folder + 'spike_times_' + str(signal_) + '.npy', spike_times)
    # np.save(save_folder + 'spike_times_thinned_' + str(signal_) + '.npy', binary_spikeTrain)
    # np.save(save_folder + 'spike_times_binned_' + str(signal_) + '.npy', ou_spiketrain_binned)

ou.shape:  (1, 1000)
(1, 20)


In [40]:
ou_poisson_rate

array([[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 1]],
      dtype=int32)

In [39]:
ou_poisson_rate_

array([[3.06455098e-01, 9.39020804e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.24378953e-01,
        1.16206049e-01, 4.12888537e-04, 8.54747962e-01, 1.10406161e+00,
        1.20108226e+00, 1.51465172e+00, 4.01128226e-01, 2.80494093e-03,
        5.02642038e-03, 6.73552719e-02, 3.58166894e-02, 1.04401235e+00]])

In [38]:
binary_spikeTrain

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1]])

In [ ]:
len(spike_times)/(10*60)

In [ ]:
np.count_nonzero(binary_spikeTrain)/(10*60)

In [ ]:
binary_spikeTrain.shape

#### Check taus

In [ ]:
def func_exp_abc_like(x, a, tau):
    return a * np.exp(-x/tau) 

disp = None # put the dispersion parameter if computed with grid-search
#maxTimeLag = 20 # only used when using autocorrelation for summary statistics
summStat_metric = 'comp_cc'
ifNorm = True # if normalize the autocorrelation or PSD
data_sumStat, data_mean, data_var, T, numTrials =  abcTau.preprocessing.extract_stats(ou_poisson_rate, deltaT, binSize,
                                                                                  summStat_metric, ifNorm, maxTimeLag=1000)

# summStat_metric = 'comp_cc'
# ifNorm = True # if normalize the autocorrelation or PSD
# ou_all_data_sumStat, ou_all_data_mean, ou_all_data_var, ou_all_T, ou_all_numTrials =  abcTau.preprocessing.extract_stats(ou_all, deltaT, binSize,
#                                                                                   summStat_metric, ifNorm, maxTimeLag)

print('data_sumStat {}, data_mean {}, data_var {}, T {}, numTrials {}'.format(data_sumStat, data_mean, data_var, T, numTrials))

ou_all_popt, ou_all_pcov = curve_fit(func_exp_abc_like, np.linspace(0,19,20), data_sumStat, maxfev=5000)
print(ou_all_popt)
ou_all_tau_ms = ou_all_popt[1] * binSize
print(ou_all_tau_ms)

In [ ]:
ou_spiketrain_binned[0].shape

In [ ]:
# Using acf func
num_lags = 20
ou_all_acf = acf(ou_spiketrain_binned[0], nlags=num_lags)
print('ou_all_acf: {}'.format(ou_all_acf))

# Using curve_fit
ou_all_popt, ou_all_pcov = curve_fit(func_exp_abc_like, np.linspace(0,20,21), ou_all_acf, maxfev=5000)

ou_all_tau_ms = ou_all_popt[1] * binSize
print(ou_all_tau_ms)

In [ ]:
# Using acf func
num_lags = 20
ou_all_acf = acf(ou_spiketrain_binned_thinned[0], nlags=num_lags)
print('ou_all_acf: {}'.format(ou_all_acf))

# Using curve_fit
ou_all_popt, ou_all_pcov = curve_fit(func_exp_abc_like, np.linspace(0,20,21), ou_all_acf, maxfev=5000)

ou_all_tau_ms = ou_all_popt[1] * binSize
print(ou_all_tau_ms)

In [ ]:
ou_all_tau_ms = ou_all_popt[1] * binSize
print(ou_all_tau_ms)